In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from PIL import Image

from tensorflow.keras.losses import MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

from pykrige.ok3d import OrdinaryKriging3D
from pykrige.uk3d import UniversalKriging3D

import h5py
from tqdm import tqdm

In [ ]:
output_dir = '/content/drive/MyDrive/Physics/Dataset10'
f = h5py.File('/content/drive/MyDrive/Physics/Physics/sst_weekly.mat', 'r')
y_test = np.load(f'{output_dir}/y_NOAA_test.npy', mmap_mode='r')

# 2D-Kriging-NOAA

In [ ]:

def generate_data(f, n, sen_num_kind_list, sen_num_var_list):
    lat = np.array(f['lat'])
    lon = np.array(f['lon'])
    sst = np.array(f['sst'])
    sst1 = np.nan_to_num(sst)
    sst_reshape = sst[0, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F')
    xv1, yv1 = np.meshgrid(lon[0, :], lat[0, :])

    X_ki = np.zeros((1040 * n, 3))
    y_ki = np.zeros((1040 * n, 1))

    for ki in tqdm(range(len(sen_num_kind_list))):
        sen_num = sen_num_kind_list[ki]
        X_va = np.zeros((1040 * n, 3))
        y_va = np.zeros((1040 * n, 1))

        for va in tqdm(range(len(sen_num_var_list))):
            times = 0
            np.random.seed(sen_num_var_list[va])

            for t in tqdm(range(1040)):
                sst_current = np.nan_to_num(sst[t, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F'))

                # Pre-calculate valid locations where SST is not NaN for selection
                valid_indices = np.argwhere(~np.isnan(sst_reshape))
                selected_indices = np.random.choice(len(valid_indices), size=sen_num, replace=False)
                sparse_locations = valid_indices[selected_indices]

                # Extract sparse data from the selected locations
                sparse_data = sst_current[sparse_locations[:, 0], sparse_locations[:, 1]]

                times += 0.00001
                X_va[n * t:n * (t + 1), :2] = sparse_locations
                X_va[n * t:n * (t + 1), 2] = times
                y_va[n * t:n * (t + 1), :] = sparse_data.reshape((n, 1))

            X_ki[1040 * n * ki:1040 * n * (ki + 1), :] = X_va
            y_ki[1040 * n * ki:1040 * n * (ki + 1), :] = y_va

    return X_ki, y_ki


In [ ]:
# Example: Data generation-2D-Kriging
n = 200

sen_num_kind_list = [n]
sen_num_var_list = [900]
X_ki, y_ki = generate_data(f, n,sen_num_kind_list, sen_num_var_list)

In [ ]:
# Grids for Kriging
gridx = np.arange(0.0, 180, 1)
gridy = np.arange(0.0, 360, 1)
gridz = np.arange(0.0, 0.00006, 0.00001)

# Sensor configurations
sen_num_kind_list = [200, 240, 280, 300, 320, 340]
sen_num_var_list = [900]  # As per your current specification

# Metrics initialization
avg_ssim_vals = {}
avg_psnr_vals = {}

# Main processing
for n in sen_num_kind_list:
    X_ki, y_ki = generate_data(f, n, sen_num_kind_list, sen_num_var_list)

    ssim_vals = []
    psnr_vals = []
    x_time = X_ki[0:3*n][:, 2]

    times = 0
    for i in range(0, 345):
        # Reshape and select subsets for Kriging
        X_va_2 = X_ki[i*n:(i+3)*n]
        y_va_2 = y_ki[i*n:(i+3)*n]

        # Perform 3D Kriging
        ok3d_2 = OrdinaryKriging3D(
            X_va_2[:, 0], X_va_2[:, 1], x_time, y_va_2,
            variogram_model="spherical", nlags=5
        )
        start_time = time.time()
        k3d1, ss3d = ok3d_2.execute("grid", gridx, gridy, gridz)
        end_time = time.time()

        times += (end_time - start_time)
        # Mask and rotate
        first_element = np.rot90(y_test[0, :, :, :1])
        mask = (first_element != 0)
        result = np.empty_like(k3d1)
        for j in range(k3d1.shape[0]):
            k3d1[j] = np.flipud(k3d1[j])
            result[j] = np.where(mask.reshape(360,180), k3d1[j], 0)
        # Metrics calculation
        true_set = [np.rot90(y_test[i+j, :, :, :1]) for j in range(3, 6)]
        result_set = result[-3:]  # Last 3 images as they correspond to the true_set

        for true, pred in zip(true_set, result_set):
            ssim_vals.append(ssim(true.squeeze(), pred, data_range=pred.max() - pred.min()))
            psnr_vals.append(psnr(true.squeeze(), pred, data_range=pred.max() - pred.min()))

    # Store average metrics
    avg_ssim = np.mean(ssim_vals)
    avg_psnr = np.mean(psnr_vals)
    avg_ssim_vals[n] = avg_ssim
    avg_psnr_vals[n] = avg_psnr

    print(f"Average SSIM for {n} sensors: {avg_ssim}")
    print(f"Average PSNR for {n} sensors: {avg_psnr}")

# 3D-Kriging

In [ ]:

def generate_data(f, n, sen_num_kind_list, sen_num_var_list):
    lat = np.array(f['lat'])
    lon = np.array(f['lon'])
    sst = np.array(f['sst'])
    sst1 = np.nan_to_num(sst)
    sst_reshape = sst[0, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F')
    xv1, yv1 = np.meshgrid(lon[0, :], lat[0, :])

    X_ki = np.zeros((1040 * n, 3))
    y_ki = np.zeros((1040 * n, 1))

    for ki in tqdm(range(len(sen_num_kind_list))):
        sen_num = sen_num_kind_list[ki]
        X_va = np.zeros((1040 * n, 3))
        y_va = np.zeros((1040 * n, 1))

        for va in tqdm(range(len(sen_num_var_list))):
            times = 0
            np.random.seed(sen_num_var_list[va])

            for t in tqdm(range(1040)):
                sst_current = np.nan_to_num(sst[t, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F'))

                # Pre-calculate valid locations where SST is not NaN for selection
                valid_indices = np.argwhere(~np.isnan(sst_reshape))
                selected_indices = np.random.choice(len(valid_indices), size=sen_num, replace=False)
                sparse_locations = valid_indices[selected_indices]

                # Extract sparse data from the selected locations
                sparse_data = sst_current[sparse_locations[:, 0], sparse_locations[:, 1]]

                times += 0.00001
                X_va[n * t:n * (t + 1), :2] = sparse_locations
                X_va[n * t:n * (t + 1), 2] = times
                y_va[n * t:n * (t + 1), :] = sparse_data.reshape((n, 1))

            X_ki[1040 * n * ki:1040 * n * (ki + 1), :] = X_va
            y_ki[1040 * n * ki:1040 * n * (ki + 1), :] = y_va

    return X_ki, y_ki

In [ ]:
# Sensor configurations and a single variable list (if it's constant)
sen_num_kind_list = [200, 240, 280, 300, 320, 340]
sen_num_var_list = [900]  # Only one variable as per your specification

# Kriging grid setup
gridx = np.arange(0.0, 180, 1)
gridy = np.arange(0.0, 360, 1)
gridz = np.arange(0.0, 0.00006, 0.00001)

# Metrics dictionaries
avg_ssim_vals = {}
avg_psnr_vals = {}

for n in sen_num_kind_list:
    # Generate data for the current sensor number
    X_ki, y_ki = generate_data(f, n, sen_num_kind_list, sen_num_var_list)
    x_time = X_ki[0:3*n][:, 2]  # Assuming x_time is extracted like this

    # Initialize metrics for each sensor configuration
    ssim_vals = []
    psnr_vals = []
    times = 0

    for i in range(0, 1):  # Just iterating once for demonstration
        X_va_2 = X_ki[i*n:(i+3)*n]
        y_va_2 = y_ki[i*n:(i+3)*n]

        # Perform 3D Kriging
        ok3d_2 = OrdinaryKriging3D(
            X_va_2[:, 0], X_va_2[:, 1], x_time, y_va_2,
            variogram_model="spherical", nlags=5
        )
        start_time = time.time()
        k3d1, ss3d = ok3d_2.execute("grid", gridx, gridy, gridz)
        end_time = time.time()

        # Metrics calculation and masking
        print(f"Inference Time for i={i}: ", end_time - start_time, "seconds")
        times += (end_time - start_time)

        first_element = np.rot90(y_test[0, :, :, :1])
        mask = (first_element != 0)
        result = np.empty_like(k3d1)
        for j in range(k3d1.shape[0]):
            k3d1[j] = np.flipud(k3d1[j])
            result[j] = np.where(mask.reshape(360,180), k3d1[j], 0)
        
        true_set = [np.rot90(y_test[i+j, :, :, :1]) for j in range(3, 6)]
        result_set = result[-3:]  # Last 3 images as they correspond to the true_set

        for true, pred in zip(true_set, result_set):
            ssim_vals.append(ssim(true.squeeze(), pred, data_range=pred.max() - pred.min()))
            psnr_vals.append(psnr(true.squeeze(), pred, data_range=pred.max() - pred.min()))

    # Calculate average metrics
    avg_ssim = np.mean(ssim_vals)
    avg_psnr = np.mean(psnr_vals)
    avg_ssim_vals[n] = avg_ssim
    avg_psnr_vals[n] = avg_psnr

    print(f"Average SSIM for {n} sensors: {avg_ssim}")
    print(f"Average PSNR for {n} sensors: {avg_psnr}")

# Close the file
f.close()
